# Preprocessing
**This notebook serves as documentation of the Preprocessing implementation. The whole process is automated in the `nextbike.preprocessing.Preprocessor` class.**

In [ ]:
!pip3 install -e ..

In [ ]:
from nextbike.preprocessing import Preprocessor
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import contextily as ctx
import numpy as np
import requests
import io
%matplotlib inline

# Data exploration

In [ ]:
# Load the data set directly as GeoDataFrame for later geo-based cleansing
preprocessor = Preprocessor()
preprocessor.load_gdf()
gdf = preprocessor.gdf
gdf.head()

The dataset contains many duplicates. Bikes cannot have two bookings at the same time. Remove them.

In [ ]:
old_row_number = len(gdf)
gdf.drop_duplicates(subset=['b_number', 'datetime'], inplace=True)
print('Number of duplicate rows removed:', old_row_number - len(gdf))

Print information about the columns.

In [ ]:
gdf.info(null_counts=True)

Inspect the NaN values for p_number.

In [ ]:
gdf[gdf.isna().any(axis=1)].sample(5)

The `p_number` seems to be `NaN` while it was intended to be `0`. This is typically the case if `p_spot` is `False` which might mean that the booking is not at a known station or the validation failed due to a technical mistake. This is probably the case for the entries where `p_name != BIKE {number}`.

Fill the values with `0`.

In [ ]:
gdf.fillna(0, inplace=True)

Now, inspect the different possible values for columns of interest.

In [ ]:
for col in ['p_spot', 'p_number', 'p_place_type', 'trip', 'b_bike_type', 'p_bike']:
    print(gdf[col].value_counts())
    print('---------------------------------')

`p_spot`, `p_place_type` and `p_bike` seem to have influence on each other according to their distribution.

**Assumption**
* `p_spot == True` means that it is a station trip. In this case `p_bike == False` because a station trip is the opposite of a free-floating trip.
* `p_place_type == 0` would mean station trip and `p_place_type == 12` free-floating trip in consequence

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(16, 6), dpi=300)
gdf['p_spot'].value_counts().plot(ax=ax1, kind='bar', title='p_spot')
gdf['p_place_type'].value_counts().plot(ax=ax2, kind='bar', title='p_place_type')
gdf['p_bike'].value_counts().plot(ax=ax3, kind='bar', title='p_bike')
fig.tight_layout()

Now, try to validate the assumption.

`p_place_type` is always `12` if and only if `p_bike == True`.

In [ ]:
gdf[gdf['p_place_type'] == 12]
gdf[(gdf['p_bike'] == True) & (gdf['p_place_type'] == 12)]

`p_bike` is always `True` if `p_spot == False`.

In [ ]:
gdf[(gdf['p_bike'] == True) & (gdf['p_spot'] == False)]

But `p_bike` can also be `False` if `p_spot == False`. **So our hypothesis cannot be validated.**

In [ ]:
gdf[(gdf['p_bike'] == False) & (gdf['p_spot'] == False)]

Neither `p_number` nor `p_uid` is a unique identifier for `p_name`

In [ ]:
print('p_number:', len(gdf['p_number'].drop_duplicates()))
print('p_uid:', len(gdf['p_uid'].drop_duplicates()))
print('p_name:', len(gdf['p_name'].drop_duplicates()))

* **The data set shows columns with prefixes p and b. What do you think do they represent? Also try to find good assumptions for the meanings of the columns**
    * `p_`: Place related information
    * `b_`: Bike related information
    * `p_spot`: True if the place is a known station, False if free-floating (with some noise)
    * `p_place_type`: 0 if the place is a known station, 12 if free-floating (with some noise)
    * `datetime`: Date of the booking
    * `b_number`: Unique identifier for a bike
    * `trip`:
        * 'first': Indicates the first booking of a day for a bike
        * 'last': Indicates the last booking of a day for a bike
        * 'start': Indicates the start of a trip
        * 'end': Inidicates the end of a trip
    * `p_uid`: ID of the location (even though no unique identifier)
    * `p_bikes`: Available bikes at the place
    * `p_lat`: Latitude of the location
    * `b_bike_type`: Type of the bike (the meaning is not clear, probably different bike versions)
    * `p_name`: Name of the location
    * `p_number`: Number of the location (even though no unique identifier)
    * `p_lng`: Longitude of the location
    * `p_bike`: True if free-floating, False if known station (with some noise)

* **The trip column in your data set shows different values. Explain why there are not only two. Are examples with certain values for trip more informative for the analysis of mobility patterns than others?**
    * The start/end trips are more informative in order to calculate the duration of a trip and to obtain the target data format

The dataset contains not only trips from Mannheim but also other cities like Marburg or Heidelberg.

**Idea:** Try to filter the data set by station trips at stations in Mannheim or bikes which match the `b_number` of bikes which are in Mannheim.

This information can be retrieved from https://mannheim.opendatasoft.com/explore/dataset/free_bike_status.

In [ ]:
s = requests.get('https://mannheim.opendatasoft.com/explore/dataset/free_bike_status/download/?format=csv&timezone=Europe/Berlin&lang=de&use_labels_for_header=true&csv_separator=%3B', verify=False).content
stations_df = pd.read_csv(io.StringIO(s.decode('utf-8')), delimiter=';')
stations_df.head()

Extract a set of distinct bike_numbers which are currently used in Mannheim.

**Problem**: There might be old bikes in the bookings which are not used anymore or bikes from Mannheim which drove to another city which is allowed according to Nextbike's policy.

In [ ]:
bike_numbers = set()
for number_list in stations_df['Fahrradnummern']:
    if isinstance(number_list, str):
        for num in number_list.split(','):
            bike_numbers.add(num)

Create a filtered data set according to station and `b_number`.

In [ ]:
filtered_df = gdf[(gdf['p_uid'].isin(stations_df['uid'])) | (gdf['b_number'].isin(bike_numbers))].reset_index(drop=True)
filtered_df.head()

**Same problem as before:** start and end trip number does not match. This is probably because start trips leaving a Mannheim station are kept but the corresponding end trip was cut (or vice versa).

In [ ]:
filtered_df['trip'].value_counts()

Filtering with the `b_number` and stations does not work properly because trips outside of Mannheim (e.g. free-floating) are still present. It might be better to filter via the GeoJson boundary of Mannheim.

In [ ]:
# Load the boundary of Mannheim as GeoJson shape and plot the real stations vs the filtered trip locations
mannheim_boundary_gdf = gpd.read_file('../data/input/mannheim_boundary.geojson', crs='EPSG:4326')
stations_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(stations_df['lng'], stations_df['lat']), crs='EPSG:4326')
filtered_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(filtered_df['p_lng'], filtered_df['p_lat']), crs='EPSG:4326')

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 16), dpi=300)

mannheim_boundary_gdf.plot(ax=ax, alpha=.6, edgecolor='blue')
filtered_gdf.plot(ax=ax, label='Filtered trip locations', marker='x', c='red')
stations_gdf.plot(ax=ax, label='Real stations', c='midnightblue')
ctx.add_basemap(ax=ax, crs='EPSG:4326')

ax.set_title('Mannheim: Real stations vs. filtered trip locations')
ax.legend()
plt.show()

**New idea:** Exlude all trips outside of the Mannheim Polygon.

In [ ]:
mannheim_boundary_polygon = mannheim_boundary_gdf['geometry'][0]
mannheim_boundary_polygon

In [ ]:
geo_filtered_gdf = gdf[gdf.within(mannheim_boundary_polygon)]
geo_filtered_gdf

Now all trips are really within Mannheim.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 4), dpi=300)

mannheim_boundary_gdf.plot(ax=ax, alpha=.6, edgecolor='blue')
geo_filtered_gdf.plot(ax=ax, label='Trips in Mannheim', c='red', marker='x')
ctx.add_basemap(ax=ax, crs='EPSG:4326')

ax.set_title('Mannheim: Trips within the city')
ax.legend()
plt.show()

As expected, there is still a mismatch between start and end trips.

In [ ]:
geo_filtered_gdf['trip'].value_counts()

# Data cleansing

To get a clean data set where each start trip has a corresponding end trip it is necessary to identify start trips without an end trip and end trips without a start trip. This is the case if two or more start trips or respectively two or more end trips occur after each other in the data set.

To be more explicit about the ordering of the data and to make sure that no wrong rows are removed the data is sorted by `b_number` and within each group of `b_number` by `datetime`.

This makes sure that for each bike the ordering of trips follows the time in which they occured.

In [ ]:
geo_filtered_gdf.sort_values(by=['b_number', 'datetime'], inplace=True)

Trips of type `first` or `last` are not relevant to detect start and end trips in our case. Remove them.

In [ ]:
geo_filtered_gdf = geo_filtered_gdf[(geo_filtered_gdf['trip'] != 'first') & (geo_filtered_gdf['trip'] != 'last')].reset_index(drop=True)
geo_filtered_gdf

**How to identify wrong start or end trips?**
Since our trips are not sorted by bike and time it is easy to detect trips without the corresponding start or end booking.
* If two consecutive rows are of trip type `start` remove the first of them
    * Reason: Before the next start booking an end booking must occur because the trips are sorted by time for each bike so the first row must be the one without corresponding end booking.
* If two consecutive rows are of trip type `end` remove the last of them
    * Reason: Two end bookings after one another mean that the second booking has no corresponding start booking because the trips are sorted by time for each bike
    
**What happens if the first booking of a bike is of type `end` and the last booking of the previous bike of type `start`?**
* This is a special case and covered by the algorithm. All trips per bike have to end with an end trip and have to start with a start trip

In [ ]:
# Define a sliding window O(N) algorithm which cleans the data set by the following scheme:
# if two consecutive rows have the same trip type:
#   if the trip type is 'start':
#     delete the first row of the two rows (because the end trip for the observed start trip is missing)
#   else:
#     delete the second row (because the start trip for the observed end trip is missing)
def fix(df):
    # Use numpy to execute the code in the Cython space
    trips = np.array(df['trip'])
    b_numbers = np.array(df['b_number'])
    # Use a hash set for distinct O(1) insertion operations
    delete_indices = set()
    # Iterate until the second last index because the sliding window is constructed by the interval [i, i + 1]
    for i in range(len(trips) - 1):
        # Special case: The trips of one bike should not end with a trip of type 'start'
        # and the booking of the next bike should not start with a trip of type 'end'
        if trips[i] == 'start' and trips[i + 1] == 'end' and b_numbers[i] != b_numbers[i + 1]:
            delete_indices.add(i)
            delete_indices.add(i + 1)
        if trips[i] == trips[i + 1]:
            i_delete = i if trips[i] == 'start' else i + 1
            delete_indices.add(i_delete)
    # Call pandas' internal drop method once in the end to hand over the execution to Cython again
    return df.drop(delete_indices, axis=0)
    
geo_cleaned_gdf = fix(geo_filtered_gdf)
geo_cleaned_gdf

Now each start trip has a corresponding end trip.

In [ ]:
geo_cleaned_gdf['trip'].value_counts()

This is the number of rows removed.

In [ ]:
len(geo_filtered_gdf) - len(geo_cleaned_gdf)

The new data looks good now!

In [ ]:
sample_size = 20
random_start = np.random.randint(sample_size, len(geo_cleaned_gdf) - sample_size)
geo_cleaned_gdf[random_start:random_start + sample_size]

# Data transformation

Split the cleaned data frame into start trips and end trips.

In [ ]:
start_gdf = geo_cleaned_gdf[geo_cleaned_gdf['trip'] == 'start'].reset_index(drop=True)
start_gdf.head(5)

In [ ]:
end_gdf = geo_cleaned_gdf[geo_cleaned_gdf['trip'] == 'end'].reset_index(drop=True)
end_gdf.head()

Each start trip has its corresponding end trip at the same index.

In [ ]:
len(start_gdf[end_gdf['b_number'] != start_gdf['b_number']])

Create an empty GeoDataFrame.

In [ ]:
transformed_gdf = gpd.GeoDataFrame(crs='EPSG:4326')

Now, use the data of the splitted data frames to calucate the target data frame.

In [ ]:
transformed_gdf['bike_number'] = start_gdf['b_number']
transformed_gdf['start_time'] = start_gdf['datetime']
transformed_gdf['weekend'] = start_gdf['datetime'].dt.dayofweek // 5 == 1
transformed_gdf['start_position'] = start_gdf['geometry']
transformed_gdf['duration'] = end_gdf['datetime'] - start_gdf['datetime']
transformed_gdf['end_time'] = end_gdf['datetime']
transformed_gdf['end_position'] = end_gdf['geometry']

In [ ]:
transformed_gdf.sample(5)

In [ ]:
transformed_gdf.info()

# Aggregate statistics for `duration`

In [ ]:
transformed_gdf['duration'].describe()

In [ ]:
duration_seconds = np.array(transformed_gdf['duration'].dt.seconds)
duration_weekend_seconds = np.array(transformed_gdf[transformed_gdf['weekend'] == True]['duration'].dt.seconds)
duration_weekday_seconds = np.array(transformed_gdf[transformed_gdf['weekend'] == False]['duration'].dt.seconds)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, dpi=300, figsize=(4, 3))
ax.bar(x='Overall', height=duration_seconds.mean())
ax.bar(x='Weekend', height=duration_weekend_seconds.mean())
ax.bar(x='Weekday', height=duration_weekday_seconds.mean())
ax.set_title('Mean of duration in seconds')
ax.set_ylabel('Seconds')
ax.set_xlabel('Scope')
plt.show()